In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [30]:
df1 = pd.read_csv("melb_data.csv")
df1.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000,S,Biggin,3/12/2016,2.5,3067,...,1,1.0,202,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000,S,Biggin,4/02/2016,2.5,3067,...,1,0.0,156,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000,SP,Biggin,4/03/2017,2.5,3067,...,2,0.0,134,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000,PI,Biggin,4/03/2017,2.5,3067,...,2,1.0,94,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000,VB,Nelson,4/06/2016,2.5,3067,...,1,2.0,120,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [31]:
df1.shape

(13580, 21)

In [32]:
df1.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [33]:
df2 = df1.drop(['Address', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'Propertycount', 'Date', 'Postcode'],axis='columns')
df2.head(3)

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,NaN,NaN,-37.7996,144.9984
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.0,1900.0,-37.8079,144.9934
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.0,1900.0,-37.8093,144.9944


# Data cleaning

## Handle NA values

In [34]:
df2.isnull().sum()

Suburb             0
Rooms              0
Type               0
Price              0
Distance           0
Bedroom2           0
Bathroom           0
Car               62
Landsize           0
BuildingArea    6450
YearBuilt       5375
Lattitude          0
Longtitude         0
dtype: int64

In [35]:
df2.shape

(13580, 13)

### Filling 'Car' nan values with 0, coz from what I think, nan for car parking means there is no parking space

In [36]:
df3 = df2.copy()
df3['Car'] = df3['Car'].fillna(0)
df3.isnull().sum()

Suburb             0
Rooms              0
Type               0
Price              0
Distance           0
Bedroom2           0
Bathroom           0
Car                0
Landsize           0
BuildingArea    6450
YearBuilt       5375
Lattitude          0
Longtitude         0
dtype: int64

### Use KNN to fill out the missing values for building area and year built

In [37]:
from sklearn.impute import KNNImputer

# Create the imputer
imputer = KNNImputer(n_neighbors=5)

df4 = df3.copy()

# Use the imputer to fill the missing values in the 'BuildingArea' and 'YearBuilt' columns
df4[['BuildingArea', 'YearBuilt']] = imputer.fit_transform(df4[['BuildingArea', 'YearBuilt']])

# Check the number of null values in each column
df4.isnull().sum()

Suburb          0
Rooms           0
Type            0
Price           0
Distance        0
Bedroom2        0
Bathroom        0
Car             0
Landsize        0
BuildingArea    0
YearBuilt       0
Lattitude       0
Longtitude      0
dtype: int64

### Remove 0 values from Landsize and BuildingArea

In [38]:
df4 = df4[(df4['Landsize'] != 0) & (df4['BuildingArea'] != 0)]

## Feature engineering

In [39]:
df4.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,151.96765,1964.684217,-37.7996,144.9984
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.00000,1900.000000,-37.8079,144.9934
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.00000,1900.000000,-37.8093,144.9944
3,Abbotsford,3,h,850000,2.5,3,2,1.0,94,151.96765,1964.684217,-37.7969,144.9969
4,Abbotsford,4,h,1600000,2.5,3,1,2.0,120,142.00000,2014.000000,-37.8072,144.9941


### Round down the yearbuilt, since there is no 1964.684217 in real life

In [40]:
import math

df4['YearBuilt'] = df4['YearBuilt'].apply(lambda x: math.floor(x))
df4.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,151.96765,1964,-37.7996,144.9984
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.00000,1900,-37.8079,144.9934
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.00000,1900,-37.8093,144.9944
3,Abbotsford,3,h,850000,2.5,3,2,1.0,94,151.96765,1964,-37.7969,144.9969
4,Abbotsford,4,h,1600000,2.5,3,1,2.0,120,142.00000,2014,-37.8072,144.9941


In [41]:
df4.Suburb.unique()

array(['Abbotsford', 'Airport West', 'Albert Park', 'Alphington',
       'Altona', 'Altona North', 'Armadale', 'Ascot Vale', 'Ashburton',
       'Ashwood', 'Avondale Heights', 'Balaclava', 'Balwyn',
       'Balwyn North', 'Bentleigh', 'Bentleigh East', 'Box Hill',
       'Braybrook', 'Brighton', 'Brighton East', 'Brunswick',
       'Brunswick West', 'Bulleen', 'Burwood', 'Camberwell', 'Canterbury',
       'Carlton North', 'Carnegie', 'Caulfield', 'Caulfield North',
       'Caulfield South', 'Chadstone', 'Clifton Hill', 'Coburg',
       'Coburg North', 'Collingwood', 'Doncaster', 'Eaglemont',
       'Elsternwick', 'Elwood', 'Essendon', 'Essendon North', 'Fairfield',
       'Fitzroy', 'Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris',
       'Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton', 'Hampton East',
       'Hawthorn', 'Heidelberg Heights', 'Heidelberg West', 'Hughesdale',
       'Ivanhoe', 'Kealba', 'Keilor East', 'Kensington', 'Kew',
       'Kew East', 'Maidstone', 'Malvern', 

### Categorize Suburb that has less than 10 houses into 'other'

In [42]:
suburb_stats = df4.Suburb.value_counts()
suburb_stats

Suburb
Reservoir         337
Bentleigh East    240
Preston           230
Richmond          196
Brunswick         187
                 ... 
Bacchus Marsh       1
Healesville         1
Bullengarook        1
Rockbank            1
Monbulk             1
Name: count, Length: 312, dtype: int64

In [43]:
len(suburb_stats[suburb_stats<=10])

111

In [44]:
df5 = df4.copy()
df5.Suburb = df5.Suburb.apply(lambda x: 'other' if x in suburb_stats[suburb_stats<=10] else x)
len(df5.Suburb.unique())

202

In [45]:
df5.Suburb.value_counts()

Suburb
other             513
Reservoir         337
Bentleigh East    240
Preston           230
Richmond          196
                 ... 
Melton             12
Keysborough        11
Oakleigh East      11
Clayton South      11
Tullamarine        11
Name: count, Length: 202, dtype: int64

In [46]:
df5.head(3)

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,151.96765,1964,-37.7996,144.9984
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.00000,1900,-37.8079,144.9934
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.00000,1900,-37.8093,144.9944


### We need to check if all the values in every columns is a float

In [47]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [48]:
def check_float(df, cols):
    for col in cols:
        if col in df.columns:
            if not df[col].apply(is_float).all():
                print(f"Column {col} non float")
            else:
                print(f"Column {col} float")
        else:
            print(f"Column {col} does not exist in the DataFrame.")

# Specify the columns you want to check
cols_to_check = ['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']
check_float(df5, cols_to_check)

Column Rooms float
Column Price float
Column Distance float
Column Bedroom2 float
Column Bathroom float
Column Car float
Column Landsize float
Column BuildingArea float
Column YearBuilt float


### Adding price per square feet

In [49]:
df5.head(3)

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,151.96765,1964,-37.7996,144.9984
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.00000,1900,-37.8079,144.9934
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.00000,1900,-37.8093,144.9944


In [50]:
df6 = df5.copy()
df6['PricePerSqFt'] = df6['Price'] / df6['Landsize']

In [51]:
df6.head(3)

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,PricePerSqFt
0,Abbotsford,2,h,1480000,2.5,2,1,1.0,202,151.96765,1964,-37.7996,144.9984,7326.732673
1,Abbotsford,2,h,1035000,2.5,2,1,0.0,156,79.00000,1900,-37.8079,144.9934,6634.615385
2,Abbotsford,3,h,1465000,2.5,3,2,0.0,134,150.00000,1900,-37.8093,144.9944,10932.835821


### Divide the 'Type' for individual prediction
- Type: h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse

In [52]:
df6.Type.value_counts()

Type
h    9264
u    1388
t     972
Name: count, dtype: int64

In [53]:
df_h = df6[df6['Type'] == 'h']
df_h.Type.value_counts()

Type
h    9264
Name: count, dtype: int64

In [54]:
df_u = df6[df6['Type'] == 'u']
df_u.Type.value_counts()

Type
u    1388
Name: count, dtype: int64

In [55]:
df_t = df6[df6['Type'] == 't']
df_t.Type.value_counts()

Type
t    972
Name: count, dtype: int64

In [56]:
df_h.to_csv('htype.csv', index=False)
df_u.to_csv('utype.csv', index=False)
df_t.to_csv('ttype.csv', index=False)